In [1]:
%load_ext autoreload
%autoreload 2

Clone the repo and install stuffs so it works on Google Collab. To run it Google Collab, uncomment everything in the cell below. To run it in your local machine, comment it

In [2]:
# !git clone https://github.com/longxvu/Adaptive_CA
# from google.colab import files
# import os
# import sys

# # To find local version of the library
# sys.path.append(os.path.join(".", 'Adaptive_CA'))
# %cd Adaptive_CA

# # Install necessary library
# !pip install openai

In [3]:
from assistant import GPTAssistant
from utils import show_json, list_all_assistant, get_api_key, generate_question_configuration
from tool_functions import display_quiz_function_json, generate_feedback_function_json
from openai import OpenAI
import pandas as pd
import json

## Init client and choosing our previously created assistant

In [4]:
# Create client and retrieve all current assistant
client = OpenAI(
    api_key=get_api_key()
)   # Maybe client should be outside assistant class but this will do for now

In [5]:
list_all_assistant(client, concise=True)

In [6]:
# # update the function that assistant will use
# client.beta.assistants.update(assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8", tools=[{"type": "function", "function": display_quiz_function_json}, {"type": "function", "function": generate_feedback_function_json}])
# client.beta.assistants.update(assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8", instructions="You will be presented with a couple of dialogue between two cartoon characters."
#                                                                                           "You need to identify a few science concepts presented in the dialogue to assist the children in learning "
#                                                                                           "the concepts.")
# client.beta.assistants.update(assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8", model="gpt-3.5-turbo-0125")
# client.beta.assistants.update(assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8", model="gpt-4-turbo")

In [27]:
assistant = GPTAssistant(client, assistant_id="asst_ZNz4lbi6z8bpKkE6APKdrpQ8")

In [28]:
print(assistant.assistant.id)
print(assistant.thread.id)
print(assistant.get_all_messages())

## Get dialogue data from xlsx file

In [29]:
dialogue_file_path = "transcripts/town_picnic_base.xlsx"
df = pd.read_excel(dialogue_file_path)

In [30]:
transcripts = df[df["text"].notnull()]["text"].tolist()
base_question = df[df["text"].notnull()]["base_q"].tolist()

In [31]:
print(transcripts[0][0:500] + "...")
print("Base question:", base_question[0])

## Main GPT dialogue loop

In [22]:
# Can set it to run forever but currently restricting to a limited amount of step for testing
max_step = 1
# num_questions = 3
step = 0
cur_dialogue_num = 0

difficulty = ["easy", "medium", "hard"]
difficulty_weights = [2, 5, 10]
question_configurations = generate_question_configuration(3, difficulty_weights)
current_difficulty = 5
max_score_per_question = 10
question_configurations

In [23]:
# Debug tool run
# assistant.wait_on_run()
# tool_calls = assistant.last_run.required_action.submit_tool_outputs.tool_calls
# for tool_call in tool_calls:
#     print(json.loads(tool_call.function.arguments))

In [24]:
# Debug step taken using the tool
# run_steps = client.beta.threads.runs.steps.list(
#     thread_id=assistant.thread.id, run_id=assistant.last_run.id, order="asc"
# )

# for idx, step in enumerate(run_steps.data):
#     print(f"{idx+1})")
#     step_details = step.step_details
#     print(json.dumps(show_json(step_details), indent=4))

In [25]:
# assistant.resolve_run_required_action()
# assistant.get_all_messages()

First, the assistant is presented with the dialogue then it is asked to summarize it. Then we ask it to create a quiz of varying difficulties about the science concept presented in the dialogue

In [26]:
while step < max_step and cur_dialogue_num < len(transcripts):
    # Current question difficulty level:
    current_difficulty_configurations, max_possible_score = question_configurations[current_difficulty][:-1], question_configurations[current_difficulty][-1]
    print("=====")
    diff_config = [f"{num_q} {diff}" for diff, num_q in zip(difficulty, current_difficulty_configurations)]
    diff_config = ",".join(diff_config)
        
    print(f"Difficulty level: {current_difficulty}. Current question configurations: {diff_config}")
    # Two steps: First, summarize the dialogue (for easier testing, can be omitted). Then ask the question about summarized dialogue
    assistant.submit_message(f"Summarize this dialogue: {transcripts[cur_dialogue_num]}.")
    print(assistant.get_last_response())
    print("----------------------------")
    assistant.submit_message(f"""
    Generate 3 questions ({diff_config}) to ask the child about the science concept presented in the dialogue. Please don't generate feedback for the quiz here.
    """)
    
    assistant.wait_on_run()
    # if assistant.run_requires_action():
    f_result = assistant.resolve_run_required_action()
    num_questions, student_responses = f_result["display_quiz"]
    print("----------------------------")
    print(f"Question by difficulty: {num_questions}")
    # print(assistant.get_last_response())
    # Get feedback
    assistant.wait_on_run()
    assistant.submit_message(f"Here are the student's answer: {student_responses}, generate a feedback for the student.")
    f_result = assistant.resolve_run_required_action()
    scores, concepts = f_result["generate_feedback"]
    assistant.wait_on_run()
    print(f"Scores: {scores}")

    # Here we calculate weighted scores based on the difficulty of each question
    # print(num_questions)
    # num_questions: (2, 0, 1), scores: [15, 0, 7]. (1st question easy: score 10, 2nd question easy: score: 5, 3rd question hard: score 7)
    print("----------------------------")
    print("Scoreboard")
    for idx, num_question in enumerate(num_questions):
        if num_question > 0:
            for score in scores[idx]:
                print(f"{difficulty[idx]}: {score}")
    print("----------------------------")
    
    weighted_scores = [sum(score) / (max_score_per_question * num_q) if num_q != 0 else 0 for num_q, score in zip(num_questions, scores)]
    weighted_scores = sum([w_score * num_q * difficulty_weights[idx] for idx, (w_score, num_q) in enumerate(zip(weighted_scores, num_questions))])
    normalized_score = weighted_scores / max_possible_score  # Normalized score for current difficulty level
    print(f"Weighted/Max: {weighted_scores}/{max_possible_score}, normalized: {normalized_score}")
    score_range, difficulty_delta = [0.2, 0.4, 0.6, 0.8, 1], [-2, -1, 0, 1, 2]
    for max_score_range, delta in zip(score_range, difficulty_delta):
        if normalized_score <= max_score_range:
            current_difficulty += delta
            break
    current_difficulty = max(0, min(current_difficulty, len(question_configurations)))  # clamp the value to 0 -> max difficulty
    print(f"Next difficulty is set to: {current_difficulty}")

    step += 1
    cur_dialogue_num += 1
    print("==============================================================================")

In [155]:
# assistant.submit_message("Generate a feedback of the answered quiz")
# assistant.resolve_run_required_action()
# assistant.wait_on_run()
# assistant.get_last_response()

In [24]:
# for msg in assistant.get_all_messages():
#     print(msg)

In [106]:
# generate_question_configuration(3)